<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0416/blob/main/tibame_0430_night.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import urllib.request as req
import bs4 as bs

def save_post_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())
    # 作者/看版/標題/日期
    metas = html.find_all("span", {"class":"article-meta-value"})
    uid_text = metas[0].text
    board_text = metas[1].text
    title_text = metas[2].text
    time_text = metas[3].text
    # print(uid_text, board_text, title_text, time_text, sep="\n")
    # 每個推虛文 [{"type":"", "id":"", "text":"", "date_ip":""}]
    # JSON格式: {"左者": "看板": "表提": "推虛文":[]}
    push_list = []
    pushes = html.find_all("div", {"class":"push"})
    for push in pushes:
        push_meta = push.find_all("span")
        # 推噓處理
        push_type_text = push_meta[0].text
        if "推" in push_type_text:
            push_type_text = 1
        elif "噓" in push_type_text:
            push_type_text = -1
        else:
            push_type_text = 0
        # 推噓id
        push_id_text = push_meta[1].text
        # 推噓內容
        push_content_text = push_meta[2].text.replace(": ", "")
        # 推噓ipdate
        push_ipdate_text = push_meta[3].text.strip()
        # print(push_type_text, push_id_text, push_content_text, push_ipdate_text, sep="\t")
        push_data = {
            "type":push_type_text,
            "id":push_id_text,
            "text":push_content_text,
            "date_ip":push_ipdate_text
        }
        push_list.append(push_data)
    # print(push_list)
    post = {
        "作者":uid_text,
        "看板":board_text,
        "標題":title_text,
        "日期":time_text,
        "推噓文":push_list,
    }
    # print(post)
    fn = url.split("/")[-1] + ".json"
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(post, f, ensure_ascii=False, indent=4)



```
當你發現你沒有class/id做篩選的時候
1. 找他老爸
2. 自己想辦法篩選
```



In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

def save_post_img(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    # prepare directory
    dn = url.split("/")[-1]
    if not os.path.exists(dn):
        os.makedirs(dn)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())

    allow_sub = ["jpg", "jpeg", "png", "gif"]
    link_list = html.find_all("a")
    for link in link_list:
        link_href = link["href"]
        sub = link_href.split(".")[-1]
        # lower是字串的專屬功能
        if sub.lower() in allow_sub:
            img_url = link_href
            # 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
            r = req.Request(img_url)
            r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
            img_resp = req.urlopen(r)
            img_content = img_resp.read()
            # 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
            # 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
            #                              wb rb -> for 非純文字
            fn = dn + "/" + img_url.split("/")[-1]
            with open(fn, "wb") as f:
                f.write(img_content)

In [ ]:
# os.path這件事我們後面會解釋
import os
# 如果資料夾不存在, 就創起來
if not os.path.exists("a/b"):
    os.makedirs("a/b")

img_url = "https://i.imgur.com/S20cVi5.jpeg"
# 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
r = req.Request(img_url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
img_resp = req.urlopen(r)
img_content = img_resp.read()
# 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
# 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
#                              wb rb -> for 非純文字
with open("a/b/test.png", "wb") as f:
    f.write(img_content)

In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/index4000.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

div_list = html.find_all("div", {"class":"title"})
for div in div_list:
    post_link = div.find("a")
    if not post_link == None:
        post_link_url = "https://www.ptt.cc" + post_link["href"]
        print(post_link_url)
        save_post_meta(post_link_url)
        save_post_img(post_link_url)

In [ ]:
# 如果你要把雲端硬碟當成隨身碟
dn = "drive/MyDrive/tiba0430"
if not os.path.exists(dn):
    os.makedirs(dn)
fn = dn + "/" + "a.txt"
with open(fn, "w", encoding="utf-8") as f:
    f.write("abcde")

In [ ]:
!zip -r /content/file.zip /content/M.1745582404.A.011.html

In [27]:
import glob
import json
import pandas as pd
# [{}, {}, {}]
table = []
for fn in glob.glob("M.*.json"):
    with open(fn, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
        table.append(data)

df = pd.json_normalize(table)
df.to_csv("ptt.csv", encoding="utf-8")

```
0. 中文分析: 分詞 我喜歡你-> 我/喜歡/你
抓出關鍵詞:
1. 出現次數: 越高, 越重要
2. 平常出現的頻率: 越低, 越重要

詞彙重要性(TF-IDF): 次數(TF) * 1/平常出現頻率(IDF)
```

In [28]:
news = """美國總統川普今天在密西根州慶祝他就職百日。在他經濟因應之道引發質疑的此時，他承諾他的總統任期才剛起步，並提及包括輝達和台積電的企業都在美投資。

綜合路透社、法新社等外媒報導，在抵達塞夫里治空軍國民兵基地（Selfridge Air National Guard Base）時，川普發表簡短談話，先是宣傳自己行政團隊在國防上的投資，並讚頌他2017至2021年第一個總統任期的外交政策紀錄。

他說國防部長赫格塞斯（Pete Hegseth）表現「十分傑出」，但未提及不久前曝光、有關赫格塞斯與私下與熟人討論高度敏感軍事資訊的事件。

共和黨籍的川普並相對罕見地展現跨黨派風度，盛讚密西根州民主黨籍州長惠特梅爾（Gretchen Whitmer），說她在「拯救」位於底特律（Detroit）東北方的塞夫里治基地上扮演關鍵角色。當地媒體去年報導，這處基地的未來令人擔憂。

川普在數十名軍人及惠特梅爾、赫格塞斯面前說：「我將支持破紀錄的1兆美元國防投資。」

他在演說中表示，塞夫里治基地將接收21架波音（Boeing）F-15X戰機。惠特梅爾則是發布聲明表示，這一步不僅確保這座基地的使命，也是密西根州「莫大的跨黨派勝利」，因為這保障了許多工作。"""

In [29]:
import jieba.analyse

jieba.analyse.extract_tags(news)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.846 seconds.
DEBUG:jieba:Loading model cost 0.846 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['塞夫',
 '密西根州',
 '投資',
 '塞斯',
 '基地',
 '總統',
 '報導',
 '赫格',
 '黨籍',
 '跨黨',
 '里治',
 '任期',
 '提及',
 '防部',
 '美國',
 '統川普',
 '慶祝',
 '經濟',
 '因應',
 '引發']

tf-idf柿子
os.path
requests